In [ ]:
def extract_polygon_features(polygon):
    # Check if the input geometry is a Polygon or a MultiPolygon
    if isinstance(polygon, Polygon):
        largest_polygon = polygon
    elif isinstance(polygon, MultiPolygon):
        # For MultiPolygon, select the largest polygon by area
        largest_polygon = max(polygon.geoms, key=lambda x: x.area)
    else:
        raise ValueError("Unsupported geometry type")

    # Calculate the number of sides for the largest polygon
    num_sides = len(largest_polygon.exterior.coords) - 1

    # Calculate the bounding box and aspect ratio for the largest polygon
    minx, miny, maxx, maxy = largest_polygon.bounds
    width = maxx - minx
    height = maxy - miny
    aspect_ratio = width / height

    # Calculate the area and perimeter for the largest polygon
    area = largest_polygon.area
    perimeter = largest_polygon.length

    # Calculate the compactness for the largest polygon
    compactness = (4 * math.pi * area) / (perimeter ** 2)

    # Return the calculated features as a dictionary
    return {
        'num_sides': num_sides,
        'aspect_ratio': aspect_ratio,
        'area': area,
        'perimeter': perimeter,
        'compactness': compactness
    }

In [ ]:
# Apply the extract_polygon_features function to the 'geometry' column
extracted_features = lup['geometry'].apply(extract_polygon_features)

# Convert the resulting Series of dictionaries to a DataFrame
features_df = pd.DataFrame(extracted_features.tolist())

# Reset index of the lup DataFrame and drop the old index column
lup.reset_index(drop=True, inplace=True)

# Concatenate the DataFrames along axis 1 (columns)
lup_with_features = pd.concat([lup, features_df], axis=1)